In [ ]:
import numpy as np
from gammapy.stats import WStatCountsStatistic

import astropy.units as u
import matplotlib.pyplot as plt

from numpy import load
import zipfile

%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 20
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['grid.linestyle'] = 'dashed' 

In [ ]:
def draw_errorbars(ax, h, color):
    y = h[0]
    binEdges = h[1]
    
    bin_centers = 0.5*(binEdges[1:]+binEdges[:-1])
    menStd     = np.sqrt(y)
    width      = 0.05

    ax.errorbar(
    bin_centers,
    y,
    yerr = y**0.5,
    xerr=0,
    marker = '.',
    ls = '',
    color = color
)
    
    return ax, bin_centers

## Read phases

In [ ]:
(zipfile.ZipFile("data/phases.zip","r")).extractall("data")
phases_vs_energy_flat = load('data/phases.npy')
#phases_vs_energy_flat = load('data/phases_performance_efficiency0.7.npy')
phases_vs_energy_flat_for_plotting = np.concatenate([phases_vs_energy_flat, phases_vs_energy_flat + 1])

## Time

In [ ]:
obs_time = 123130.63 * u.s # seconds

## Significance calculation

In [ ]:
# ON region P1: 
phases_on_p1 = [0.983, 0.026]
p1_duty_cycle = (1 - phases_on_p1[0]) + phases_on_p1[1]

# ON region P2:
phases_on_p2 = [0.377, 0.422]
p2_duty_cycle = phases_on_p2[1] - phases_on_p2[0]

# OFF region:
phase_off = [0.520, 0.870]
off_duty_cycle = phase_off[1] - phase_off[0]

# Number of ON/OFF
Non_p1 = len(phases_vs_energy_flat[(phases_vs_energy_flat < phases_on_p1[1]) 
                                | (phases_vs_energy_flat > phases_on_p1[0])])
Non_p2 = len(phases_vs_energy_flat[(phases_vs_energy_flat < phases_on_p2[1]) 
                                & (phases_vs_energy_flat > phases_on_p2[0])])
Noff = len(phases_vs_energy_flat[(phases_vs_energy_flat < phase_off[1]) 
                                & (phases_vs_energy_flat > phase_off[0])])

######## P1 ######### 
alpha_p1 = p1_duty_cycle / off_duty_cycle
Nex_p1 = Non_p1 - alpha_p1 * Noff
Significance_p1 = Nex_p1 / np.sqrt(alpha_p1 * Noff)

stat_p1 = WStatCountsStatistic(n_on=Non_p1, n_off=Noff, alpha=alpha_p1)
significance_p1_lima = stat_p1.sqrt_ts

######## P2 ######### 
alpha_p2 = p2_duty_cycle / off_duty_cycle
Nex_p2 = Non_p2 - alpha_p2 * Noff
Significance_p2 = Nex_p2 / np.sqrt(alpha_p2 * Noff)

stat_p2 = WStatCountsStatistic(n_on=Non_p2, n_off=Noff, alpha=alpha_p2)
significance_p2_lima = stat_p2.sqrt_ts

######## Joint ######### 
stat_joint = WStatCountsStatistic(n_on=Non_p1 + Non_p2 , n_off=Noff , alpha=alpha_p1 + alpha_p2)
significance_joint_lima = stat_joint.sqrt_ts


#########################################################################################
textstr = r'Events = {:.0f} '\
            f'\n'\
            r'Time = {:.1f}'\
            f'\n'\
            r'N$_{{\rm ex}}$ (P1) = {:.0f} '\
            f'\n'\
            r'Significance (P1) = {:.1f} $\sigma$ '\
            f'\n'\
            r'N$_{{\rm ex}}$ (P2) = {:.0f} '\
            f'\n'\
            r'Significance (P2) = {:.1f} $\sigma$'\
            f'\n'\
            r'$\bf{{Significance\ (Joint)}}$ $\bf{{= {:.1f} \sigma}}$'.format(len(phases_vs_energy_flat),
                                                      obs_time.to(u.h),
                                                      Nex_p1,
                                                      significance_p1_lima,
                                                      Nex_p2,
                                                      significance_p2_lima,
                                                      significance_joint_lima)

props = dict(boxstyle='round', facecolor='wheat', alpha=0.95)
########################################################################################

print(textstr)
print(f'Non_p1 {Non_p1}')
print(f'Non_p2 {Non_p2}')
print(f'Noff {Noff}')
print(f'Noff * alpha_p1 {Noff * alpha_p1}')
print(f'Noff * alpha_p2 {Noff * alpha_p2}')

print(f'alpha_p1 {alpha_p1}')
print(f'alpha_p2 {alpha_p2}')

In [ ]:
def format_axes(ax):
    # OFF regions
    ax.axvspan(0, phases_on_p1[1], ymin=0., ymax=1., alpha=0.4, color='salmon', hatch='/')
    ax.axvspan(phases_on_p1[0], 1 + phases_on_p1[1], ymin=0., ymax=1., alpha=0.4, color='salmon', hatch='/')
    ax.axvspan(phases_on_p2[0], phases_on_p2[1], ymin=0., ymax=1., alpha=0.4, color='salmon', hatch='/')
    ax.axvspan(phases_on_p1[0] + 1, 1.00 + 1, ymin=0., ymax=1., alpha=0.4, color='salmon', hatch='/')
    ax.axvspan(phases_on_p2[0] + 1, phases_on_p2[1] + 1, ymin=0., ymax=1., alpha=0.4, color='salmon', hatch='/')

    # ON regions
    ax.axvspan(phase_off[0], phase_off[1], ymin=0., ymax=1., alpha=0.2, color='gray', hatch='\\')
    ax.axvspan(phase_off[0] + 1, phase_off[1] + 1, ymin=0., ymax=1., alpha=0.2, color='gray', hatch='\\')

    ax.set_xlim(0, 2)
    ax.set_xlabel(f'Pulsar phase [$\phi$]')
    ax.set_ylabel(f'Events')

In [ ]:
def fill_excesses(ax, hphase, bin_centers, off_average):
    
    ## Fill Nexcesses
    #P1
    bin_width = hphase[1][1] - hphase[1][0]
    ax.bar(bin_centers[((bin_centers < phases_on_p1[1]) | (bin_centers > phases_on_p1[0])) & (bin_centers < 1)], 
       hphase[0][((bin_centers < phases_on_p1[1]) | (bin_centers > phases_on_p1[0])) & (bin_centers < 1)] - off_average, 
       bottom = off_average, width = bin_width, color = '#ff022d')
    ax.bar(bin_centers[((bin_centers < phases_on_p1[1] + 1) | (bin_centers > phases_on_p1[0]  + 1)) & (bin_centers > 1)], 
       hphase[0][((bin_centers < phases_on_p1[1]  + 1) | (bin_centers > phases_on_p1[0]  + 1)) & (bin_centers > 1)] - off_average, 
       bottom = off_average, width = bin_width, color = '#ff022d')
    #P2
    ax.bar(bin_centers[(bin_centers < phases_on_p2[1] ) & (bin_centers > phases_on_p2[0] )], 
       hphase[0][(bin_centers < phases_on_p2[1]) & (bin_centers > phases_on_p2[0])] - off_average, 
       bottom = off_average, width = bin_width, color = '#ff022d')
    ax.bar(bin_centers[(bin_centers < phases_on_p2[1] + 1) & (bin_centers > phases_on_p2[0]  + 1)], 
       hphase[0][(bin_centers < phases_on_p2[1]  + 1) & (bin_centers > phases_on_p2[0]  + 1)] - off_average, 
       bottom = off_average, width = bin_width, color = '#ff022d')

    return ax

In [ ]:
nbins = 60
fig, ax = plt.subplots(figsize = (30,10))
plt.rcParams['font.size'] = 30


format_axes(ax)

hphase = ax.hist(phases_vs_energy_flat_for_plotting, 
                 bins = nbins * 2, range=(0,2), color='dodgerblue')
hphase = ax.hist(phases_vs_energy_flat_for_plotting, histtype='step', lw=2,
                 bins = nbins * 2, color = 'k', range=(0,2))
ax, bin_centers = draw_errorbars(ax, hphase, 'k')


# OFF average
histogram_bin_size = hphase[1][1] - hphase[1][0]
off_normalization = histogram_bin_size / off_duty_cycle
off_average = off_normalization * Noff
ax.hlines(y=off_average, xmin=0, xmax=2, ls='--', color = 'k', lw=2)

fill_excesses(ax, hphase, bin_centers, off_average)

# Limits
max_min_diff = (np.max(hphase[0]) - np.min(hphase[0]))
ax.set_ylim(np.min(hphase[0]) - max_min_diff * 1/6, np.max(hphase[0]) +  max_min_diff * 1/6)
ax.text(0.015, 0.95, 'P1', fontsize=24, color='salmon', transform=ax.transAxes)
ax.text(0.515, 0.95, 'P1', fontsize=24, color='salmon', transform=ax.transAxes)
ax.text(0.215, 0.95, 'P2', fontsize=24, color='salmon', transform=ax.transAxes)
ax.text(0.715, 0.95, 'P2', fontsize=24, color='salmon', transform=ax.transAxes)

ax.text(0.75, 0.98, textstr, transform=ax.transAxes, fontsize=24,
        verticalalignment='top', bbox=props)
plt.tight_layout()

!mkdir -p figures
fig.savefig('figures/Phaseogram_filled_no_title_landscape.png')
fig.savefig('figures/Phaseogram_filled_no_title_landscape.pdf')